# DATA CURATION

This script will serve to curate our two dataset: one which deals with the politicians by country provided by the Wikimedia API, another is a world population dataset provided by the Population Reference Bureau.

In [34]:
# First, let's import the modules that we'll be using to curate the data

import pandas as pd
import numpy as np
import string
import os

In [35]:
# Now we'll read in our data

demographics_by_case = pd.read_csv("raw_data/covid19_case_demographics.csv")

economic_data = pd.read_excel("raw_data/MOSLURN.xls", skiprows = 10)
stlouis_cases_over_time = pd.read_csv("raw_data/City-of-St-Louis-COVID-19-Case-Data.csv")
metrics_by_county = pd.read_csv("raw_data/MetricsbyCounty.csv")
demographics = pd.read_csv("raw_data/demographics.csv")

We'll need a dataframe that contains the population/cases for the 10 most populous counties in Missouri. Let's first modify our metrics by county dataset to only reflect the following counties: St. Louis, Jackson, St. Charles, St. Louis City, Greene, Jefferson, Boone, Jasper, Cass

In [36]:
counties = ['ST LOUIS', 'JACKSON', 'ST CHARLES', 'ST LOUIS CITY', 'GREENE', 'CLAY', 'JEFFERSON', 'BOONE', 'JASPER', 'CASS']

st_louis_metrics = metrics_by_county[metrics_by_county['County'].isin(counties) ]

st_louis_metrics.head(20)

,County,Measure Names,7-Day Range End Date,7-Day Range Start Date,Dashboard Update Date,Most Recent Test Date,Measure Values
135,BOONE,Population,11/6/2021,10/31/2021,11/9/2021,11/8/2021,"180,463"
136,BOONE,* Positive PCR Cases,11/6/2021,10/31/2021,11/9/2021,11/8/2021,"22,496"
137,BOONE,* Positive Antigen Cases,11/6/2021,10/31/2021,11/9/2021,11/8/2021,"2,702"
138,BOONE,Confirmed Deaths,11/6/2021,10/31/2021,11/9/2021,11/8/2021,144
139,BOONE,PCR Tests,11/6/2021,10/31/2021,11/9/2021,11/8/2021,"284,956"
140,BOONE,Antigen Tests,11/6/2021,10/31/2021,11/9/2021,11/8/2021,"55,991"
141,BOONE,* 7 Day Range (Recent) - PCR Cases,11/6/2021,10/31/2021,11/9/2021,11/8/2021,136
142,BOONE,* 7 Day Range (Recent) - Antigen Cases,11/6/2021,10/31/2021,11/9/2021,11/8/2021,25
143,BOONE,7-Day Confirmed Deaths,11/6/2021,10/31/2021,11/9/2021,11/8/2021,0
144,BOONE,7-Day PCR Tests,11/6/2021,10/31/2021,11/9/2021,11/8/2021,"2,671"


We'll save a subset of this dataframe that reflects the total number of cases in these counties over the course of the pandemic

In [37]:
columns = ['Population', '* Positive PCR Cases']

cases = st_louis_metrics[st_louis_metrics['Measure Names'].isin(columns)]

cases.to_csv('final_data/Missouri_cases_10_county.csv')

We'll save another subset of this dataframe that reflects the total number of deaths in the counties over the course of the pandemic

In [38]:
columns = ['Population', 'Deaths']

deaths = st_louis_metrics[st_louis_metrics['Measure Names'].isin(columns)]

deaths.to_csv('final_data/Missouri_deaths_10_county.csv')

Next We'll subset our economic dataframe to reflect the time period of unemployment during the course of the COVID-19 pandemic

NOTE: The MOSLURN attribute measures the unemployment rate in the county for a given month

In [53]:
covid_econ_data = economic_data[economic_data['observation_date'] >= '2020-03-01' ]

covid_econ_data.head()

,observation_date,MOSLURN
362,2020-03-01,4.0
363,2020-04-01,11.5
364,2020-05-01,10.3
365,2020-06-01,9.0
366,2020-07-01,7.9


In [42]:
# Let's save this daframe to a csv file that we'll use later
covid_econ_data.to_csv('final_data/stlouis_covid_econ_data.csv')

Now we'll create a dataframe that has the hospitalizations by demographic in St. Louis County. To do this, we'll merge our hospitalizations_by_county and demographics dataframe using a location key, then subsetting the resulting dataframe for St. Louis County only

NOTE: 29189 is the FIPS code for St. Louis County, MO 

In [40]:
data = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/epidemiology.csv")

In [41]:
data = data[data['location_key'] == 'US_MO']

data.head(20)

,date,location_key,new_confirmed,new_deceased,new_recovered,new_tested,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested
8241435,2020-01-22,US_MO,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
8241436,2020-01-23,US_MO,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
8241437,2020-01-24,US_MO,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
8241438,2020-01-25,US_MO,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
8241439,2020-01-26,US_MO,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
8241440,2020-01-27,US_MO,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
8241441,2020-01-28,US_MO,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
8241442,2020-01-29,US_MO,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
8241443,2020-01-30,US_MO,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
8241444,2020-01-31,US_MO,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN


In [18]:
# Now we'll write this dataframe to a csv file to use for our analysis

data.to_csv('final_data/Missouri_case_rate_aggregate.csv')

Now we'll look which demographics were impacted the most by covid

In [56]:
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/rolling-averages/us-counties-2021.csv'
df = pd.read_csv(url)

st_louis = df[(df['county'] == 'St. Louis') & (df['state'] == 'Missouri')]

st_louis.head(20)

,date,geoid,county,state,cases,cases_avg,cases_avg_per_100k,deaths,deaths_avg,deaths_avg_per_100k
1651,2021-01-01,USA-29189,St. Louis,Missouri,0,456.57,45.92,0,8.14,0.82
4910,2021-01-02,USA-29189,St. Louis,Missouri,922,508.43,51.14,1,7.14,0.72
8169,2021-01-03,USA-29189,St. Louis,Missouri,597,554.14,55.74,0,7.14,0.72
11428,2021-01-04,USA-29189,St. Louis,Missouri,448,569.71,57.30,0,7.14,0.72
14687,2021-01-05,USA-29189,St. Louis,Missouri,454,525.00,52.81,14,7.14,0.72
17946,2021-01-06,USA-29189,St. Louis,Missouri,0,525.00,52.81,2,3.43,0.34
21205,2021-01-07,USA-29189,St. Louis,Missouri,1392,544.71,54.79,48,9.29,0.93
24464,2021-01-08,USA-29189,St. Louis,Missouri,554,545.88,54.91,10,9.38,0.94
27723,2021-01-09,USA-29189,St. Louis,Missouri,898,620.43,62.40,9,11.86,1.19
30982,2021-01-10,USA-29189,St. Louis,Missouri,455,600.14,60.36,0,11.86,1.19


Similar to what we did previously, lets extend this dataframe to include a couple of rural counties to determine whether individuals living in more rural counties are more or less at risk than people living in urban counties such as St. Louis

In [65]:
counties = ['St. Louis', 'Boone', 'Jackson', 'St. Charles']

counties_df = df[df['state'] == 'Missouri']

urban_v_rural = counties_df[counties_df['county'].isin(counties)]

urban_v_rural.head(20)

,date,geoid,county,state,cases,cases_avg,cases_avg_per_100k,deaths,deaths_avg,deaths_avg_per_100k
1651,2021-01-01,USA-29189,St. Louis,Missouri,0,456.57,45.92,0,8.14,0.82
1654,2021-01-01,USA-29183,St. Charles,Missouri,0,204.13,50.77,0,2.88,0.72
1704,2021-01-01,USA-29095,Jackson,Missouri,254,162.43,42.38,2,2.14,0.56
1742,2021-01-01,USA-29019,Boone,Missouri,0,87.57,48.53,0,0.75,0.42
4910,2021-01-02,USA-29189,St. Louis,Missouri,922,508.43,51.14,1,7.14,0.72
4913,2021-01-02,USA-29183,St. Charles,Missouri,309,194.20,48.31,0,2.88,0.72
4963,2021-01-02,USA-29095,Jackson,Missouri,148,167.57,43.72,0,2.14,0.56
5001,2021-01-02,USA-29019,Boone,Missouri,0,87.57,48.53,0,0.75,0.42
8169,2021-01-03,USA-29189,St. Louis,Missouri,597,554.14,55.74,0,7.14,0.72
8172,2021-01-03,USA-29183,St. Charles,Missouri,143,187.14,46.55,0,2.88,0.72


Before we save this dataframe for our analyses, let's convert the daily rates to weekly rates. This will make it easier for us to plot trendline graphs as well as do group comparisons with other counties

In [66]:
from datetime import datetime

urban_v_rural.date = pd.to_datetime(urban_v_rural.date)

weekly_data = urban_v_rural.groupby('county').resample('W-Wed', label = 'right', closed = 'right', on = 'date').sum().reset_index().sort_values(by='date')

weekly_data.head(20)

,county,date,cases,cases_avg,cases_avg_per_100k,deaths,deaths_avg,deaths_avg_per_100k
0,Boone,2021-01-06,631,574.71,318.48,2,3.86,2.16
49,Jackson,2021-01-06,1016,1067.42,278.52,17,14.85,3.88
147,St. Louis,2021-01-06,2421,3138.85,315.72,17,40.13,4.04
98,St. Charles,2021-01-06,1130,1214.76,302.16,4,13.33,3.32
148,St. Louis,2021-01-13,4700,4096.09,411.99,122,84.58,8.49
1,Boone,2021-01-13,738,788.18,436.74,3,3.07,1.71
50,Jackson,2021-01-13,1529,1377.31,359.38,19,18.42,4.80
99,St. Charles,2021-01-13,6553,1522.35,378.69,2,7.91,1.94
51,Jackson,2021-01-20,964,1210.00,315.72,11,15.65,4.09
100,St. Charles,2021-01-20,1260,1667.29,414.73,8,7.43,1.84


In [67]:
# we'll write this dataframe to csv file that we'll use in our subsequent analyses

weekly_data.to_csv('final_data/MO_stlouis_v_rural.csv')

Now we'll adjust our demographic dataframes for practical use in our analyses

In [47]:
demographics_by_case = demographics_by_case.drop('OBJECTID', axis = 1)

demographics_by_case.head()

,group,cases,rate_per_100k
0,Male,58920,12440.6
1,Female,71441,13605.9
2,9 and younger,8207,6919.8
3,10 to 19,15910,12494.8
4,20 to 29,22149,17474.0


In [49]:
demographics_by_deaths = demographics_by_deaths.drop('OBJECTID', axis = 1)

demographics_by_deaths.head()

,group,deaths,deaths_per_100k
0,Male,1234,260.6
1,Female,1322,251.8
2,39 and younger,17,3.4
3,40 to 49,53,44.4
4,50 to 59,143,100.1


In [50]:
# Now we'll write both dataframes to csv files

demographics_by_deaths.to_csv('final_data/demographics_by_deaths.csv')
demographics_by_case.to_csv('final_data/demographics_by_case.csv')